In [ ]:
#to be written in regulated format, pay attention to <space>
#Aalto_EngineeringForHumans, 2 Fitts' Law GUI, 25.6.2017
#for empirical data fitting, need to install Plotly package, install guidance: https://plot.ly/python/getting-started/
import tkinter as tk
import plotly.plotly as pl
import time as tm

#input parameters
dValue = 100
wValue = 10
global counter
counter = 0
global timeStamp
timeStamp = 0
global timeStampTemp
timeStampTemp = 0
global timeDelta
timeDelta = 0

root = tk.Tk()
root.title("Fitts' Law: Empirical Experiment GUI")
#experiment GUI
frameExplaination = tk.Frame(root)
frameIdController = tk.Frame(root)
framePointingArea = tk.Frame(root, borderwidth=2, relief=tk.GROOVE, width=400)
frameModelling = tk.Frame(root, width=100)
frameExplaination.grid(row=0, column=0, columnspan=2)
frameIdController.grid(row=1, column=0, columnspan=2)
frameModelling.grid(row = 2, column = 0, padx = 10, pady = 10)
framePointingArea.grid(row=2, column=1, padx=10, pady=10)

#explaination text
textExplaination=tk.Label(frameExplaination,text="explaination text here")
textExplaination.grid(row=0,column=0)
#ID controller
scaleIdController=tk.Scale(frameIdController,from_=2,to=10,orient=tk.HORIZONTAL)
scaleIdController.grid(row=0,column=0)
#Pointing area
frameObj1=tk.Frame(framePointingArea,width=wValue,height=100)
frameObj2=tk.Frame(framePointingArea,width=wValue,height=100)
frameDistance=tk.Frame(framePointingArea,width=dValue,height=100)
framePlaceHolder=tk.Frame(framePointingArea,width=10,height=30)
obj1=tk.Button(frameObj1,bg='#224147')
obj2=tk.Button(frameObj2,bg='#224147')
labelCounter=tk.Label(framePointingArea,text="counter = %d" %counter)
frameObj1.grid(row=0,column=0)
frameDistance.grid(row=0,column=1)
frameObj2.grid(row=0,column=2)
framePlaceHolder.grid(row=1,column=0)
obj1.grid(row=0,column=0,sticky=tk.EW)
obj2.grid(row=0,column=0,sticky=tk.EW)
labelCounter.grid(row = 1, column = 1)

def cursorMovement(event):
    global timeStamp
    global timeStampTemp
    global counter
    event.widget.config(bg='#224147')
    #timing cursor one-way movement time
    timeStamp = tm.clock()
    timeDelta = timeStamp - timeStampTemp
    timeStampTemp = timeStamp
    tk.Label(frameModelling,text="time = %0.2f s" %timeDelta, anchor = tk.W).grid(row = 1 + counter, column=0, sticky=tk.N)
    #counting cursor back-and-forth movement number of times
    counter += 1
    labelCounter.config(text="counter = %d" %(counter-1))

#when cursor hover over button, button change colour
obj1.bind("<Enter>", lambda event, h=obj1: h.configure(bg='#6fccdd'))
obj1.bind("<Leave>", cursorMovement)
obj2.bind("<Enter>", lambda event, h=obj2: h.configure(bg='#6fccdd'))
obj2.bind("<Leave>", cursorMovement)

#empirical data fitting


root.mainloop()
